Install Lib

In [1]:
pip install python-chess

     ---------------------------------------- 0.0/156.5 kB ? eta -:--:--
     -- ------------------------------------- 10.2/156.5 kB ? eta -:--:--
     --------- --------------------------- 41.0/156.5 kB 495.5 kB/s eta 0:00:01
     ------------------------- ---------- 112.6/156.5 kB 939.4 kB/s eta 0:00:01
     -------------------------------------- 156.5/156.5 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148535 sha256=b35fff96328dddac3f6776a528a3dbdf019125f2677b4e45b1af2a6859e6ace4
  Stored in directory: c:\users\dnilson\appdata\local\pip\cache\wheels\84\8b\6c\94b05653eadf8766c01d0ccc56e866c570346c5ea36b3db6f3
Su


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Class Node

In [14]:
board = chess.Board()

In [18]:
board.turn == chess.WHITE

True

In [19]:
############ CLASS NODE ############

from typing import Optional, List, Tuple
import random
import chess
import chess.polyglot
import copy

class Node:
    def __init__(self, board: chess.Board, parent: Optional['Node'] = None, from_move: Optional[chess.Move] = None):
        self.board = board
        self.parent = parent
        self.from_move = from_move
        self.children: List['Node'] = []
        self.visits = 0
        self.value = 0
        self.used_moves = set()  # Rastreamento dos movimentos já usados

    def is_fully_expanded(self) -> bool:
        return len(self.children) == len(list(self.board.legal_moves))

    def update(self, reward: int):
        self.visits += 1
        if reward == 1: # checkmate for white
            if self.board.turn == chess.BLACK:
                self.value += 1
        elif reward == -1:
            if self.board.turn == chess.BLACK:
                self.value += -1
        elif reward == 0.5: # draw
            self.value += 0
        if self.parent:
            self.parent.update(-reward)

    def expand(self) -> Optional['Node']:
        legal_moves = list(self.board.legal_moves)
        available_moves = [move for move in legal_moves if move not in self.used_moves]
        while available_moves:
            move = random.choice(available_moves)
            self.used_moves.add(move)  # Marcar o movimento como usado
            available_moves.remove(move)
            new_board = copy.deepcopy(self.board)
            new_board.push(move)
            leaf = Node(new_board, parent=self, from_move=move)
            self.children.append(leaf)

### MONTE CARLO TREE SEARCH

In [20]:
import math

def uct(node: Node) -> float:
    # UCT formula: Q + C * sqrt(ln(N) / n)
    if node.visits == 0:
        return float('inf')
    C = 1.414  # Exploração constante, ajustável
    return node.value / node.visits + C * math.sqrt(math.log(node.parent.visits) / node.visits)

def selection(root: Node) -> Node:
    root.expand()
    leaf = max(root.children, key=uct)
    return leaf

def expansion(node: Node) -> Node:
    if not node.is_fully_expanded():
        node.expand()
        children = max(node.children, key=uct)
        return children
    else:
        if node.children:
            children = max(node.children, key=uct)
            return children
        else:
            return node

def simulation(children: Node) -> int:
    board = copy.deepcopy(children.board)
    while not board.is_game_over():
        legal_moves = list(board.legal_moves)
        move = random.choice(legal_moves)
        board.push(move)
    result = board.result()
    if result == '1-0':
        reward = 1
    elif result == '0-1':
        reward = -1
    else:
        reward = 0.5
    return reward

def backpropagation(reward: int, node: Node) -> None:
    node.update(reward)

def best_child(root: Node, player: bool) -> Node:
    best_child = random.choice(root.children)
    best_visits = 0
    for child in root.children:
        if child.visits > best_visits:
            best_visits = child.visits
            best_child = child
    return best_child

def mcts(board: chess.Board, n_simulations: int, player: bool) -> Optional[chess.Move]:
    root = Node(board)  # Inicialize a raiz da árvore fora do loop
    for _ in range(n_simulations):
        leaf = selection(root)  # Seleciona
        node = expansion(leaf)  # Expande
        reward = simulation(node)  # Simula
        backpropagation(reward, node)  # Retropropaga
    best = best_child(root, player)
    move = best.from_move
    return move

### GAME

In [21]:
############ GAME ############
import chess
import chess.pgn

def play_game():

    board = chess.Board()
    # initial_fen = board.fen()  # Capturar a posição inicial do tabuleiro

    # initial_fen = "6k1/4Q2N/4K3/8/8/8/8/8 w - - 0 1" # checkmate white
    initial_fen =  "6K1/4q2n/4k3/8/8/8/8/8 w - - 0 1" # checkmate black
    board.set_fen(initial_fen)

    print(board)
    move_history = []

    # Salvar o histórico dos movimentos em formato PGN
    game = chess.pgn.Game()
    game.headers["Event"] = "Test"
    game.headers["White"] = "MCTS"
    game.headers["Black"] = "MCTS"
    game.headers["SetUp"] = "1"
    game.headers["FEN"] = initial_fen

    # Jogo
    while not board.is_game_over():
        if board.turn == chess.WHITE:
            move = mcts(board, 1600, board.turn)
            print(f"Player WHITE move: {board.san(move)}")
        else:
            move = mcts(board, 1600, board.turn)
            print(f"Player BLACK move: {board.san(move)}")

        move_history.append(board.san(move))
        board.push(move)

        if board.is_checkmate():
            print(f'Player {"WHITE" if board.turn == chess.BLACK else "BLACK"} Wins!')
            game.headers["Result"] = board.result()
            break
        elif board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition():
            print('Draw!')
            game.headers["Result"] = board.result()
            break

    # Salvar o histórico dos movimentos em formato PGN
    node = game
    board = chess.Board()  # Resetar o tabuleiro para a posição inicial
    board.set_fen(initial_fen)  # Redefinir a posição inicial

    for move in move_history:
        move = board.parse_san(move)  # Converter SAN para objeto Move
        node = node.add_main_variation(move)
        board.push(move)  # Atualizar o tabuleiro com o movimento

    # Imprimir o histórico de movimentos
    print("\nHistórico de movimentos:")
    print(game)

play_game()

. . . . . . K .
. . . . q . . n
. . . . k . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
Player WHITE move: Kh8
Player BLACK move: Qh4
Player WHITE move: Kg8
Player BLACK move: Kf6
Player WHITE move: Kh8
Player BLACK move: Kf7
Draw!

Histórico de movimentos:
[Event "Test"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "MCTS"]
[Black "MCTS"]
[Result "1/2-1/2"]
[SetUp "1"]
[FEN "6K1/4q2n/4k3/8/8/8/8/8 w - - 0 1"]

1. Kh8 Qh4 2. Kg8 Kf6 3. Kh8 Kf7 1/2-1/2
